<center> <img src = "https://raw.githubusercontent.com/denis-marchenkov-sf/assets/aed651ad29de045160567af37cd221ec604c6baa/hh%20label.jpg" alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [753]:
import pandas as pd
import psycopg2
import plotly.express as px
import requests
from bs4 import BeautifulSoup

# отключим назойливые предупреждения о pandas коннекторе
import warnings
warnings.filterwarnings('ignore', message = '.*SQLAlchemy connectable')

In [754]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
# TODO: remove
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [755]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Количество вакансий (таблица vacancies).

In [756]:
# текст запроса
query_3_1 = f'''

SELECT
    COUNT(*)
FROM
    public.vacancies

'''

In [757]:
# результат запроса
df_vacancy_total = pd.read_sql_query(query_3_1, connection)

print('Количество вакансий:')
display(df_vacancy_total)


Количество вакансий:


,count
0,49197


2. Количество работодателей (таблица employers). 

In [758]:
# текст запроса
query_3_2 = f'''

SELECT
    COUNT(*)
FROM
    public.employers

'''

In [759]:
# результат запроса
df_employer_total = pd.read_sql_query(query_3_2, connection)

print('Количество работодателей:')
display(df_employer_total)

Количество работодателей:


,count
0,23501


3. Количество регионов (таблица areas).

In [760]:
# текст запроса
query_3_3 = f'''

SELECT
    COUNT(*)
FROM
    public.areas

'''

In [761]:
# результат запроса
df_region_total = pd.read_sql_query(query_3_3, connection)

print('Количество регионов:')
display(df_region_total)

Количество регионов:


,count
0,1362


4. Количество сфер деятельности (таблица industries).

In [762]:
# текст запроса
query_3_4 = f'''

SELECT
    COUNT(*)
FROM
    public.industries

'''

In [763]:
# результат запроса
df_industry_total = pd.read_sql_query(query_3_4, connection)

print('Количество сфер деятельности:')
display(df_industry_total)


Количество сфер деятельности:


,count
0,294


# Выводы по предварительному анализу данных
Всего в базе представлено 49197 вакансий по 1362 регионам в 294 сферах деятельности от 23501 работодателя.

***

# Юнит 4. Детальный анализ вакансий

1. Количество вакансий в каждом регионе.

In [764]:
# текст запроса
query_4_1 = f'''

SELECT
    area.name AS area,
    COUNT(vacancy.id) AS cnt
FROM
    public.vacancies AS vacancy
JOIN
    public.areas AS area ON vacancy.area_id = area.id
GROUP BY
    area.id
ORDER BY cnt DESC

'''

In [765]:
# результат запроса
df_vacancy_region = pd.read_sql_query(query_4_1, connection)

print('Количество вакансий по регионам в порядке убывания:')
display(df_vacancy_region)

Количество вакансий по регионам в порядке убывания:


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Кизляр,1
765,Джизак,1
766,Эртиль,1
767,Арсеньев,1


2. У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [766]:
# текст запроса
query_4_2 = f'''

SELECT
    COUNT(*)
FROM
    public.vacancies
WHERE
    salary_from IS NOT NULL
    OR
    salary_to IS NOT NULL

'''

In [767]:
# результат запроса
df_vacancy_not_null = pd.read_sql_query(query_4_2, connection)

print('Количество вакансий с указанной зарплатой:')
display(df_vacancy_not_null)

Количество вакансий с указанной зарплатой:


,count
0,24073


3. Cредние значения для нижней и верхней границы зарплатной вилки округлённые до целого.

In [768]:
# текст запроса
query_4_3 = f'''

SELECT
    ROUND( AVG(salary_from) )   AS avg_from,
    ROUND( AVG(salary_to) )     AS avg_to
FROM
    public.vacancies

'''

In [769]:
# результат запроса
df_avg_salary = pd.read_sql_query(query_4_3, connection)

print('Cредние значения для нижней и верхней границы зарплатной вилки:')
display(df_avg_salary)

Cредние значения для нижней и верхней границы зарплатной вилки:


,avg_from,avg_to
0,71065.0,110537.0


4. Количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях.


In [770]:
# текст запроса
query_4_4 = f'''

SELECT
    schedule,
    employment,
    COUNT(id) AS cnt
FROM
    public.vacancies
GROUP BY
    schedule, employment
ORDER BY cnt DESC

'''

In [771]:
# результат запроса
df_schedule = pd.read_sql_query(query_4_4, connection)

print('Количество вакансий по типам рабочего графика и трудоустройства:')
display(df_schedule)

Количество вакансий по типам рабочего графика и трудоустройства:


,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Значения поля "Требуемый опыт работы" (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [772]:
# текст запроса
query_4_5 = f'''

SELECT
    experience,
    COUNT(id) AS cnt
FROM
    public.vacancies
WHERE
    experience IS NOT NULL
GROUP BY
    experience
ORDER BY cnt ASC

'''

In [773]:
# результат запроса
df_experience = pd.read_sql_query(query_4_5, connection)

print('Требуемый опыт работы:')
display(df_experience)

Требуемый опыт работы:


,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


# Выводы по детальному анализу вакансий

Москва, Санкт-Петербург, Минск, Новосибирск и Алматы входят в пятёрку лидеров по количеству вакансий.
У половины вакансий не указана заработная плата.
Средние показатели заработной платы лежат в диапазоне от 70 до 110 тысяч.
Наиболее востребованы сотрудники на полную занятость и полный рабочий день.
Свыше половины вакансий требуют наличие опыта работы от 1 года до 3 лет.

<div>
    <img src="images/vacancy_by_region.png">
</div>
<br/>
<div>
    <img src="images/salary_specified.png">
</div>
<br/>
<div>
    <img src="images/schedule_employment.png">
</div>
<br/>
<div>
    <img src="images/experience.png">
</div>

***

In [774]:
# построение графиков детального анализа вакансий

# города-лидеры по вакансиям
bar_vacancy = px.bar(
    df_vacancy_region.head(5),
    x = 'area',
    y = 'cnt',
    color = 'area',
    color_discrete_sequence = ['orange', 'yellow', 'green', 'blue', 'teal'],
    labels = { 'cnt':'Количество вакансий', 'area':'Город' },
    title = 'Распределение количества вакансий в первой пятёрке городов'
)

bar_vacancy.update_layout(
    title_x = 0.5
)

bar_vacancy.write_image("images/vacancy_by_region.png")

bar_vacancy.show()


# процент вакансий с указанной зарплатой
total_vacancies = df_vacancy_total['count'][0]
vacancies_not_null = df_vacancy_not_null['count'][0]
vacancies_null = total_vacancies - vacancies_not_null

null_percent = vacancies_null / total_vacancies * 100
not_null_percent = vacancies_not_null / total_vacancies * 100

data = [null_percent, not_null_percent]

pie_salary = px.pie(
    data,
    values = data,
    names = ['Одна из границ ЗП не указана', 'Указана хотя бы одна граница ЗП'],
    title = "Процент вакансий с указанием ЗП",
    color_discrete_sequence = ['red', 'green']
)

pie_salary.update_traces(pull = [0.1, 0])

pie_salary.write_image("images/salary_specified.png")

pie_salary.show();


# требования по опыту работы
pie_experience = px.pie(
    df_experience,
    values = 'cnt',
    names = 'experience',
    title = "Требования к наличию опыта работы",
    color_discrete_sequence = ['orange', 'yellow', 'green', 'blue']
)


pie_experience.write_image("images/experience.png")

pie_experience.show();


# взаимосвязь между рабочим графиком, типом занятости и количеством вакансий
scatter_schedule = px.scatter(
    df_schedule,
    x = 'employment',
    y = 'schedule',
    color = 'cnt',
    size = 'cnt',
    size_max = 50,
    labels = {'schedule':'График работы', 'employment':'Тип занятости', 'cnt':'Количество вакансий'},
    title = "Взаимосвязь между рабочим графиком и количеством вакансий",
)

scatter_schedule.write_image("images/schedule_employment.png")

scatter_schedule.show()



# Юнит 5. Анализ работодателей

1. Какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [775]:
# текст запроса
query_5_1 = f'''

SELECT
    employer.name,
    COUNT(vacancy.id) AS cnt
FROM
    public.vacancies AS vacancy
JOIN
    public.employers AS employer ON vacancy.employer_id = employer.id
GROUP BY
    employer.id
ORDER BY cnt DESC
LIMIT 5

'''

In [776]:
# результат запроса
df_employers = pd.read_sql_query(query_5_1, connection)

print('Топ пять работодателей по количеству вакансий:')
display(df_employers)

Топ пять работодателей по количеству вакансий:


,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Количество работодателей для каждого региона и вакансий в нём.
Среди регионов, в которых нет вакансий, найдём тот, в котором наибольшее количество работодателей.


In [777]:
# текст запроса
query_5_2 = f'''

SELECT
    area.name                AS area_name,
    COUNT(vacancy.id)        AS vacancy_count,
    COUNT(employer.id)       AS employer_count
FROM
    public.areas area
LEFT JOIN
    public.vacancies vacancy ON area.id = vacancy.area_id
LEFT JOIN
    public.employers employer ON area.id = employer.area
GROUP BY
    area.id
ORDER BY
    vacancy_count ASC, employer_count DESC

'''

In [778]:
# результат запроса

# регион без вакансий и с наибольшим количеством работодателей - первый по списку

print('Регионы по количеству вакансий и работодателей:')
df_regions = pd.read_sql_query(query_5_2, connection)
display(df_regions)

Регионы по количеству вакансий и работодателей:


,area_name,vacancy_count,employer_count
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18
...,...,...,...
1357,Новосибирск,1149438,1149438
1358,Алматы,1364132,1364132
1359,Минск,2354880,2354880
1360,Санкт-Петербург,6320667,6320667


3. Количество регионов с опубликованными вакансиями для каждого работодателя.


In [779]:
# текст запроса
query_5_3 = f'''

SELECT
    employer.name                        AS employer_name,
    COUNT(DISTINCT vacancy.area_id)      AS area_count
FROM
    public.vacancies vacancy
JOIN
    public.employers employer ON vacancy.employer_id = employer.id
GROUP BY
    employer.id
ORDER BY
    area_count DESC

'''

In [780]:
# результат запроса
print('Количество регионов с опубликованными вакансиями для каждого работодателя:')
df_regions_vacancies = pd.read_sql_query(query_5_3, connection)
display(df_regions_vacancies)

Количество регионов с опубликованными вакансиями для каждого работодателя:


,employer_name,area_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Количество работодателей, у которых не указана сфера деятельности.

In [781]:
# текст запроса
query_5_4 = f'''

SELECT
    COUNT(*)
FROM 
    public.employers employer
LEFT JOIN
    public.employers_industries employer_industry ON employer.id = employer_industry.employer_id
WHERE
    employer_industry.employer_id IS NULL

'''

In [782]:
# результат запроса
print('Количество работодателей, у которых не указана сфера деятельности:')
df_null_industry = pd.read_sql_query(query_5_4, connection)
display(df_null_industry)

Количество работодателей, у которых не указана сфера деятельности:


,count
0,8419


5. Узнаем название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [783]:
# текст запроса
query_5_5 = f'''

SELECT
    employer.name
FROM
    public.employers_industries employer_industry
JOIN
    public.employers employer ON employer_industry.employer_id = employer.id
GROUP BY
    employer.id
HAVING 
    COUNT(industry_id) = 4
ORDER BY
    employer.name
OFFSET 2
LIMIT 1

'''

In [784]:
# результат запроса
print('Третья по алфавиту компания, у которой указаны 4 сферы деятельности:')
df_four_industry = pd.read_sql_query(query_5_5, connection)
display(df_four_industry)

Третья по алфавиту компания, у которой указаны 4 сферы деятельности:


,name
0,2ГИС


6. У какого количества работодателей в качестве сферы деятельности указана "Разработка программного обеспечения".


In [785]:
# текст запроса
query_5_6 = f'''

SELECT
    COUNT(DISTINCT employer_industry.employer_id)
FROM
    public.employers_industries employer_industry
JOIN
    public.industries industry ON employer_industry.industry_id = industry.id
WHERE
    industry.name = 'Разработка программного обеспечения'

'''

In [786]:
# результат запроса
print('Количество компаний в сфере "Разработка программного обеспечения":')
df_count_software = pd.read_sql_query(query_5_6, connection)
display(df_count_software)

Количество компаний в сфере "Разработка программного обеспечения":


,count
0,3553


7. Список регионов-миллионников, в которых представлены вакансии компании «Яндекс» вместе с количеством вакансий в этих регионах.

In [787]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

cities_data = soup.select('table.standard.sortable > tbody > tr > td > a')
million_cities = [x.text for x in cities_data]

city_query = ', '.join([f'\'{x}\'' for x in million_cities])

print('Список городов-миллионников:')
print(city_query)

Список городов-миллионников:
'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград'


In [788]:
# текст запроса
query_5_7 = f'''

WITH vacancies_by_region AS
(
    SELECT
        area.name,
        COUNT(vacancy.id) AS cnt
    FROM
        public.vacancies vacancy
    JOIN
        public.employers employer ON vacancy.employer_id = employer.id
    JOIN
        public.areas area ON vacancy.area_id = area.id
    WHERE
        employer.name = 'Яндекс'
        AND
        area.name IN ({city_query})
    GROUP BY
        area.id
    ORDER BY
        cnt
)


SELECT
    *
FROM
    vacancies_by_region

UNION ALL

SELECT
    'Total',
    SUM(cnt)
FROM
    vacancies_by_region

'''

In [789]:
# результат запроса
print('Города-миллионники с вакансиями от компании "Яндекс":')
df_yandex_mil = pd.read_sql_query(query_5_7, connection)
display(df_yandex_mil)

Города-миллионники с вакансиями от компании "Яндекс":


,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Ростов-на-Дону,25.0
5,Казань,25.0
6,Пермь,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


# Выводы по анализу работодателей

***

In [790]:
# построение графиков по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [791]:
# текст запроса

In [792]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [793]:
# текст запроса

In [794]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [795]:
# текст запроса

In [796]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [797]:
# текст запроса

In [798]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [799]:
# текст запроса

In [800]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [801]:
# текст запроса

In [802]:
# результат запроса

***

In [803]:
# выводы по предметному анализу

# Общий вывод по проекту

In [804]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

In [805]:
connection.close()